In [1]:
import os
from bs4 import BeautifulSoup
import string
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.models import load_model
from keras.layers.core import Dense, Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras.utils import to_categorical
import pickle
from keras.optimizers import RMSprop
import numpy as np
from numpy import array
import heapq

2023-02-24 11:11:11.168227: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-24 11:11:11.331623: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-24 11:11:11.331645: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-24 11:11:12.362057: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

**Import data**

In [2]:
name="LSTM 64"
documents = []
for file in os.listdir("reuters_sample/"): # original: "reuters_data/"
    if file.endswith('.sgm'): # it is important for GoogleColab
        filename = os.path.join("reuters_sample", file) # original: "reuters_data"
        f = open(filename, 'r', encoding='utf-8', errors='ignore')
        dataFile = f.read().lower()
        
        soup = BeautifulSoup(dataFile, 'html.parser')

        ## get all 'topic'
        # topics = {topic.name for topic in soup.find_all()}
  
        ## iterate all 'topic'
        # for topic in topics:
          
        ## find all element of 'topic'
          # for i in soup.find_all('topic'):
  
        ## if tag has attribute of class
            # if i.has_attr('trade'):
  
# We have selected the following 20 'TOPICS' out of 135:
# that we want to use for our prediction exercise: 1.) "trade" 2.) "earn" 3.) "grain" 4.) "money-fx" 5.) 
# "coffee" 6.) "gold" 7.) "acq" 8.) "wheat" 9.) "veg-oil" 10.) "nat-gas" 11.) "cooper" 12.) "ship" 13.) 
# "dlr" 14.) "crude" 15.) "interest" 16.) "meal-feed" 17.) "alum" 18.) "money-supply" 19.) "cocoa" 20.) "livestock"
        contents = soup.findAll('title')
        
        for content in contents:
            documents.append(content.text)

In [3]:
print('Number of documents: {}'.format(len(documents)))

Number of documents: 6966


In [4]:
# Remove duplicated strings from the list of strings
documents = [i for n, i in enumerate(documents) if i not in documents[:n]]

print('Number of documents: {}'.format(len(documents)))

Number of documents: 6731


In [5]:
documents[:10]

['inco sees no major impact from dow removal',
 'former empire of carolina <emp> exec sentenced',
 'doctors find link between aids, smallpox virus',
 'birth control pills help prevent cancer - study',
 'u.s. economic data key to debt futures outlook',
 'u.s. "action program" for sub-saharan africa',
 'unusual texas instruments <txn> preferred priced',
 'clark says he expects u.s. action on acid rain',
 'ford motor <f> distributes profit sharing',
 'jamaica puts cap on borrowing']

**Join the documents**

In [6]:
data = ""
for d in documents:
    data += d

In [7]:
print('Number of data: {}'.format(len(data)))

Number of data: 347403


In [8]:
# improve punctuation
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')
print('Number of data: {}'.format(len(data)))
print(data[:100])

Number of data: 346768
inco sees no major impact from dow removalformer empire of carolina <emp> exec sentenceddoctors find


In [9]:
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
data = data.translate(translator)
print(data[:100])

inco sees no major impact from dow removalformer empire of carolina  emp  exec sentenceddoctors find


In [10]:
# integer encode text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded_data = tokenizer.texts_to_sequences([data])[0]
print(len(encoded_data))
encoded_data[:5]

51500


[1644, 19, 81, 366, 1094]

In [11]:
# determine the vocabulary size
# unique_words = tokenizer.word_index
unique_words = np.unique(encoded_data)
vocab_size = len(unique_words) + 1  # 0 is reserved for padding so that's why we added 1
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 14407


**Next, we need to create sequences of words to fit the model with one word as input and one word as output.**

In [12]:
# create word -> word sequences
WORD_LENGTH = 5
prev_words = []
next_words = []
for i in range(1, len(encoded_data) - WORD_LENGTH):
    prev_words.append(encoded_data[i:i + WORD_LENGTH])
    next_words.append(encoded_data[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

[19, 81, 366, 1094, 46]
470


In [13]:
print('Total Sequences: %d' % len(prev_words))

Total Sequences: 51494


In [14]:
# list(len(prev_words)[:5]) # [input, output]



**We can then split the sequences into input (X) and output elements (y)**



In [15]:
# split into X and y elements
X = prev_words
X = np.array(X)
Y = next_words
Y = np.array(Y)

# X = np.zeros((len(prev_words), WORD_LENGTH, vocab_size), dtype=bool)
# Y = np.zeros((len(next_words), vocab_size), dtype=bool)

In [16]:
print(X[:5])
print(Y[:5])

[[  19   81  366 1094   46]
 [  81  366 1094   46  470]
 [ 366 1094   46  470 3706]
 [1094   46  470 3706 3707]
 [  46  470 3706 3707    8]]
[ 470 3706 3707    8 1645]


In [17]:
# one hot encode outputs
Y = to_categorical(Y, num_classes=vocab_size)
# define model
Y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [18]:
X.shape
Y.shape

(51494, 14407)

**Build the model**

In [19]:
model = Sequential()
model.add(Embedding(vocab_size, 5, input_length=5)) # original: 5
model.add(LSTM(64))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

2023-02-23 22:43:29.156867: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 10)             144070    
                                                                 
 lstm (LSTM)                 (None, 64)                19200     
                                                                 
 dense (Dense)               (None, 14407)             936455    
                                                                 
Total params: 1,099,725
Trainable params: 1,099,725
Non-trainable params: 0
_________________________________________________________________
None


**Train the model**

In [20]:
# fit network
optimizer = RMSprop(learning_rate=0.01)

# compile network
#### since labels are INTEGERS, we need to changed from loss='categorical_crossentropy'!!!
#### If you want to provide labels using one-hot representation, please use CategoricalCrossentropy loss.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # optimizer ='adam'
history =model.fit(X, Y, epochs=20).history

Epoch 1/20
1610/1610 [==============================] - 41s 22ms/step - loss: 8.3509 - accuracy: 0.0310
Epoch 2/20
1610/1610 [==============================] - 26s 16ms/step - loss: 7.7608 - accuracy: 0.0428
Epoch 3/20
1610/1610 [==============================] - 37s 23ms/step - loss: 7.4973 - accuracy: 0.0541
Epoch 4/20
1610/1610 [==============================] - 50s 31ms/step - loss: 7.2161 - accuracy: 0.0702
Epoch 5/20
1610/1610 [==============================] - 47s 29ms/step - loss: 6.9066 - accuracy: 0.0845
Epoch 6/20
1610/1610 [==============================] - 47s 29ms/step - loss: 6.5791 - accuracy: 0.0964
Epoch 7/20
1610/1610 [==============================] - 46s 29ms/step - loss: 6.2337 - accuracy: 0.1083
Epoch 8/20
1610/1610 [==============================] - 44s 28ms/step - loss: 5.9019 - accuracy: 0.1195
Epoch 9/20
1610/1610 [==============================] - 45s 28ms/step - loss: 5.5890 - accuracy: 0.1327
Epoch 10/20
1610/1610 [==============================] - 44s 27m

In [ ]:
# After successful training, we will save the trained model and just load it back as needed.
model.save('keras_next_word_model'+str(name)+'.h5')
pickle.dump(history, open('history'+str(name)+'.p', 'wb'))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib


In [ ]:
#Loading back the saved weights and history

model = load_model('keras_next_word_model'+str(name)+'.h5')
history = pickle.load(open('history'+str(name)+'.p', 'rb'))

In [ ]:
#Evaluation
plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.title('model accuracy of '+str(name))
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc= 'upper left')

plt.savefig("01.Accuracy"+str(name)+".png")

In [ ]:
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('model loss of ' +str(name))
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc= 'upper left')

plt.savefig("02.Loss"+str(name)+".png")

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
max_sequence_len=6
seed_text = "computer terminal systems <cpml> completes"
next_words = 2
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    #predicted = model.predict_classes(token_list, verbose=0)
    predict_x=model.predict(token_list) 
    predicted=np.argmax(predict_x,axis=1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)